In [1]:
import pandas as pd
import numpy as np

In [2]:
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

In [3]:
service_code = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')

/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
vehicle_make = pd.read_excel('../data/Vehicle_Make.xlsx')

Additional cleaning tasks:
* Remove faults occurring in the vicinity of the service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)
* Remove faults where the EquipmentID has more than 5 characters.

Upcoming full derate indicated by an SPN 5246 <-- requires a tow. 
SPN 1569, FMI = 31 75% derate

In [68]:
faults = pd.read_csv('../data/J1939Faults.csv', index_col = 'RecordID', parse_dates = ['EventTimeStamp', 'LocationTimeStamp'], low_memory = False)

service_location1 = faults[(faults["Latitude"] == 36.066666) & (faults["Longitude"] == -86.434722)]

service_location2 = faults[(faults["Latitude"] == 35.588333) & (faults["Longitude"] == -86.443888)]

service_location3 = faults[(faults["Latitude"] == 36.1950) & (faults["Longitude"] ==  -83.174722)]

service_locations = pd.concat([service_location1, service_location2, service_location3])

faults_cleaned = faults.drop(service_locations.index)

faults_cleaned = faults_cleaned[faults_cleaned['EquipmentID'].map(len) <= 5]

faults_cleaned = faults_cleaned.drop(['actionDescription', 'faultValue', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'MCTNumber'], axis = 1).reset_index()
#consider keeping ecuSoftwareVersion. Maybe an updated software throws more codes

diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv') \
    .pivot(index = 'FaultId', columns = 'Name', values = 'Value')


faults_diagnostics = faults_cleaned.merge(diagnostics, left_on = 'RecordID', right_on = 'FaultId') \
    .set_index('EventTimeStamp').sort_index()

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
faults_cleaned['eventDescription'].nunique()

741

In [63]:
faults_diagnostics.groupby('EquipmentID').head()

,RecordID,ESS_Id,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
EventTimeStamp,,,,,,,,,,,,,,,,,,,,,
2000-03-18 19:14:10,1211417,108604425,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,...,NaN,True,127.4,1279,False,NaN,0,NaN,100,0.58
2000-03-18 19:14:10,1211418,108604426,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,...,NaN,True,127.4,1279,False,NaN,0,NaN,100,0.58
2000-03-18 19:20:47,1211419,108604487,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,...,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-18 19:20:47,1211420,108604488,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,...,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-19 02:59:58,1211422,108608408,Not Reporting Data Wheel Sensor ABS Axle 2 Right,AAAI000032*AAAM000038*BB41275 *A82J140721A_9...,5W26153559,EC80ESP,BNDWS,11,792,7,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-05 11:50:20,1248072,123708690,NaN,05572391*03002387*110719010101*62602004*N1*BKR*,80239366,6X1u20D1500000000,CMMNS,0,5842,14,...,NaN,True,125.6,18431,True,NaN,0,12.35,100,0
2020-03-05 12:15:33,1248084,123714477,NaN,05572391*03002387*110719010101*62602004*N1*BKR*,80239366,6X1u20D1500000000,CMMNS,0,5842,14,...,NaN,NaN,NaN,18431,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-05 13:31:06,1248115,123731337,NaN,05572391*03002387*110719010101*62602004*N1*BKR*,80239366,6X1u20D1500000000,CMMNS,0,5742,9,...,NaN,True,105.8,18431,True,NaN,0,12.1,100,0


In [65]:
filtered = faults_cleaned.loc[faults_cleaned['spn'].eq(5246).groupby(faults_cleaned['EquipmentID']).transform('any')]

In [69]:
derates = faults_cleaned[((faults_cleaned['spn'] == 1569) & (faults_cleaned['fmi'] == 31))| (faults_cleaned['spn'] == 5246)]

In [83]:
derates['EquipmentID'].unique()

array(['1721', '1395', '1515', '1630', '1487', '1499', '1329', '1365',
       '1419', '1486', '1509', '1492', '1340', '1328', '1430', '1370',
       '1497', '1350', '1473', '1608', '1606', '1612', '1354', '1493',
       '1583', '1465', '1335', '1789', '1383', '1389', '1501', '1438',
       '1463', '1514', '1666', '1689', '1692', '1357', '1396', '310',
       '1602', '1660', '1445', '1431', '1327', '1795', '1399', '1494',
       '1367', '1364', '1452', '1595', '1740', '1366', '1339', '1425',
       '1358', '1646', '1391', '1428', '1338', '1637', '1346', '1490',
       '1562', '1433', '1663', '1417', '1424', '1771', '1790', '1591',
       '1797', '1596', '1421', '1547', '1444', '1480', '1466', '1457',
       '1794', '1696', '1703', '1626', '1715', '1437', '1453', '1584',
       '1597', '1610', '1377', '1652', '1517', '1784', '1751', '1786',
       '1590', '1341', '1382', '1714', '1711', '1369', '1398', '1791',
       '1772', '1472', '1418', '1645', '1475', '1815', '1806', '1607',
       

In [87]:
faults_diagnostics.columns

Index(['RecordID', 'ESS_Id', 'eventDescription', 'spn', 'fmi', 'active',
       'activeTransitionCount', 'EquipmentID', 'Latitude', 'Longitude',
       'LocationTimeStamp', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus',
       'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure'],
      dtype='object')

In [67]:
filtered[filtered['EquipmentID']!=5246]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
5,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59
9,10,990446,2015-02-21 11:41:55,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,False,1,1417,105438630,33.039953,-96.182592,2015-02-21 11:41:51
10,11,990459,2015-02-21 11:42:19,Low (Severity Low) Engine Coolant Level,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55
13,14,990470,2015-02-21 11:43:40,Low (Severity Low) Engine Coolant Level,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,False,1,1582,105410980,40.161944,-80.161018,2015-02-21 11:43:36
15,16,990495,2015-02-21 11:45:06,Low (Severity Low) Engine Coolant Level,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,False,1,1601,105430420,38.272037,-78.937129,2015-02-21 11:45:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184671,1248429,123886552,2020-03-06 12:11:54,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,False,126,1853,105376821,26.079814,-80.123518,2020-03-06 12:11:49
1184673,1248431,123891846,2020-03-06 12:20:36,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17
1184674,1248432,123891847,2020-03-06 12:20:36,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17
1184680,1248438,123893761,2020-03-06 12:51:53,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,False,126,1853,105376821,26.078888,-80.122361,2020-03-06 12:51:49


In [ ]:
derates_list = ['1721', '1395', '1515', '1630', '1487', '1499', '1329', '1365',
       '1419', '1486', '1509', '1492', '1340', '1328', '1430', '1370',
       '1497', '1350', '1473', '1608', '1606', '1612', '1354', '1493',
       '1583', '1465', '1335', '1789', '1383', '1389', '1501', '1438',
       '1463', '1514', '1666', '1689', '1692', '1357', '1396', '310',
       '1602', '1660', '1445', '1431', '1327', '1795', '1399', '1494',
       '1367', '1364', '1452', '1595', '1740', '1366', '1339', '1425',
       '1358', '1646', '1391', '1428', '1338', '1637', '1346', '1490',
       '1562', '1433', '1663', '1417', '1424', '1771', '1790', '1591',
       '1797', '1596', '1421', '1547', '1444', '1480', '1466', '1457',
       '1794', '1696', '1703', '1626', '1715', '1437', '1453', '1584',
       '1597', '1610', '1377', '1652', '1517', '1784', '1751', '1786',
       '1590', '1341', '1382', '1714', '1711', '1369', '1398', '1791',
       '1772', '1472', '1418', '1645', '1475', '1815', '1806', '1607',
       '1560', '1450', '1401', '1572', '1375', '1368', '1742', '1650',
       '1443', '1495', '1670', '1587', '1778', '1566', '1617', '1691',
       '1573', '1477', '1476', '1524', '305', '1441', '1405', '1577',
       '1447', '1659', '1408', '1439', '1403', '1458', '1535', '1668',
       '1814', '1567', '1373', '1446', '1700', '1629', '1730', '1665',
       '1381', '1551', '1413', '1643', '1508', '1440', '1455', '1844',
       '1571', '1429', '1491', '1803', '1467', '1378', '1488', '1739',
       '1448', '1420', '1384', '1683', '304', '1372', '1502', '1582',
       '1464', '1599', '1712', '1407', '1603', '1644', '1757', '1618',
       '1604', '1519', '1831', '1631', '1811', '1628', '1818', '1423',
       '1636', '1555', '1719', '1638', '1710', '1823', '1435', '1808',
       '1392', '1669', '1649', '1579', '1416', '1534', '1575', '1397',
       '1539', '1510', '1813', '1601', '1657', '1621', '1651', '1620',
       '1708', '1592', '1613', '1619', '1925', '1887', '1662', '1640',
       '1881', '1563', '1713', '1557', '1839', '1833', '1605', '1513',
       '1639', '1866', '1485', '1581', '1549', '1776', '301', '308',
       '1586', '1661', '1635', '1907', '1853', '1785', '1594', '1859',
       '1600', '1963', '1848', '1918', '1561', '1796', '1624', '1598',
       '1972', '1706', '1641', '1874', '1940', '1609', '1642', '1506',
       '1552', '1685', '1883', '1946', '1913', '1920', '1634', '1995',
       '1512', '1878', '1564', '1686', '1554', '1505', '1802', '1788',
       '1868', '1807', '1688', '1922', '1903', '1681', '1529', '1904',
       '1615', '1828', '1694', '1559', '1827', '1745', '1842', '1720',
       '1525', '1970', '1877', '1888', '1989', '1537', '1932', '1521',
       '1804', '1854', '1851', '1565', '1981', '1528', '1741', '1980',
       '1737', '1580', '1924', '1822', '1766', '1812', '1917', '1933',
       '1817', '1654', '1731', '2011', '2017', '2008', '1726', '1717',
       '1656', '1543', '1835', '1664', '1880', '1916', '1996', '1912',
       '1947', '1997', '1736', '1820', '1729', '1864', '1556', '1898',
       '1884', '1832', '1705', '1585', '1908', '1752', '1623', '1800',
       '1732', '1725', '1845', '1876', '1576', '1897', '1647', '1931',
       '1971', '1852', '1697', '1959', '1953', '1787', '1962', '1893',
       '1746', '2015', '2019', '1957', '1593', '1965', '1873', '1911',
       '1934', '1942', '1982', '1900', '2009', '1948', '1723', '2006',
       '1956', '2012', '1836', '1935', '307', '1879', '1792', '1865',
       '1885', '1690', '1793', '1749', '1860', '1894', '1856', '1704',
       '1614', '1655', '1667', '1632', '1699', '1687', '1625', '1871',
       '1750', '1611', '2013', '1939', '1886', '1999', '1819', '1901',
       '1821', '1847', '1701', '1588', '1838', '1849', '1775', '1743',
       '1952', '1988', '1716', '1862', '1863', '1622', '2004', '1764',
       '1905', '1698', '1846', '309', '1798', '1780', '1762', '1979',
       '2007', '1919', '1966', '1843', '1938', '1744', '1992', '1976',
       '1926', '2143', '1718', '1899', '2111', '1978', '1763', '1870',
       '1889', '1994', '2027', '1967', '1928', '1921', '1891', '1734',
       '1748', '1945', '2010', '306', '1951', '1768', '1781', '2002',
       '1728', '1869', '1735', '1964', '1858', '1758', '1958', '2120',
       '1756', '2209', '1944', '1927', '2005', '1709', '2082', '1914',
       '2020', '1872', '1975', '1857', '1733', '1961', '1954', '1985',
       '1824', '1993', '1895', '1906', '2018', '1977', '1834', '1930',
       '1969', '1991', '1984', '1949', '1896', '2109', '2030', '1950',
       '1855', '2000', '1986', '1782', '1937', '2021', '1943', '2089',
       '1968', '1909', '2211', '1829', '302', '1861', '2175']

In [ ]:
faults_cleaned[faults_cleaned['spn']== 5246]

In [ ]:
diagnostics.pivot(index = 'FaultId' ,columns = 'Name', values = 'Value').reset_index()

In [ ]:
service_code[service_code['SPN']== 5246]

In [ ]:
service_code[service_code['SPN']== 1569]